In [6]:
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q peft
!pip install -q accelerate
!pip install -q trl
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 5.8 

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,

)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
def generate_text(model, tokenizer, prompt_text, max_tokens):
  prompt_text = tokenizer(prompt_text, return_tensors="pt")
  outputs = model.generate(
      input_ids=prompt_text["input_ids"].to(model.device),
      attention_mask=prompt_text["attention_mask"].to(model.device),
      max_new_tokens=max_tokens,
      repetition_penalty=1.5,
      eos_token_id=tokenizer.eos_token_id,
  )
  return tokenizer.decode(outputs, skip_special_tokens=True)

In [ ]:
def load_data(file_path):
    percent_of_train_dataset = 0.80
    data = load_dataset("json", data_files=file_path, split="train")

    split_dataset = data.train_test_split(
        train_size=int(data.num_rows * percent_of_train_dataset), seed=45,
        shuffle=True
    )
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    print(f"Size of the train set: {len(train_dataset)}. Size of the validation set: {len(eval_dataset)}")
    return train_dataset, eval_dataset

train_dataset, eval_dataset = load_data(file_path="/content/prescription.jsonl")

Size of the train set: 235. Size of the validation set: 59


In [ ]:
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit
from transformers import TrainingArguments
from transformers import Trainer, DataCollatorForLanguageModeling

In [7]:
from huggingface_hub import notebook_login
notebook_login()

## GEMMA

In [ ]:
model_name = "google/gemma-2-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
tuning_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.RANDOM,
    num_virtual_tokens=4,
    tokenizer_name_or_path=model_name
)

peft_model = get_peft_model(model, tuning_config)

In [ ]:
training_args = TrainingArguments(
    use_cpu=False,
    output_dir="./",
    auto_find_batch_size=True,
    learning_rate=0.005,
    num_train_epochs=5
)

In [ ]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [instruction, output]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [ ]:
train_dataset

Dataset({
    features: ['instruction', 'output'],
    num_rows: 235
})

In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [2]:
import os
from openai import OpenAI

In [4]:
os.environ["OPENAI_API_KEY"] = api_key

In [5]:
client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# CodeLlamma

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,

)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [3]:
def generate_text(model, tokenizer, prompt_text, max_tokens):
  prompt_text = tokenizer(prompt_text, return_tensors="pt")
  outputs = model.generate(
      input_ids=prompt_text["input_ids"].to(model.device),
      attention_mask=prompt_text["attention_mask"].to(model.device),
      max_new_tokens=max_tokens,
      repetition_penalty=1.5,
      eos_token_id=tokenizer.eos_token_id,
  )
  return tokenizer.decode(outputs, skip_special_tokens=True)

In [5]:
def load_data(file_path):
    percent_of_train_dataset = 0.80
    data = load_dataset("json", data_files=file_path, split="train")

    split_dataset = data.train_test_split(
        train_size=int(data.num_rows * percent_of_train_dataset), seed=45,
        shuffle=True
    )
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    print(f"Size of the train set: {len(train_dataset)}. Size of the validation set: {len(eval_dataset)}")
    return train_dataset, eval_dataset

train_dataset, eval_dataset = load_data(file_path="/content/prescription.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

Size of the train set: 235. Size of the validation set: 59


In [6]:
model_name = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
prompt = '''Your task is to extract valid JSON from the given sample and output it in 1 JSON file

{'start_index': '512', 'end_index': '520', 'text':'seizure free', 'attributes': {'LowerNumberOfSeizures': None, 'UpperNumberOfSeizures': None, 'TimePeriod': None, 'NumberOfTimePeriods': None, 'MonthDate': None, 'TimeSince_or_TimeOfEvent': 'Since', 'NumberOfSeizures': '0', 'PointInTime': 'LastClinic', 'FrequencyChange': None, 'YearDate': None, 'LowerNumberOfTimePeriods': None, 'UpperNumberOfTimePeriods': None, 'AgeUnit':



'''
max_new_tokens = 100
inputs = tokenizer.encode(prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=max_new_tokens,
                         num_return_sequences=1)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Your task is to extract valid JSON from the given sample and output it in 1 JSON file

{'start_index': '512', 'end_index': '520', 'text':'seizure free', 'attributes': {'LowerNumberOfSeizures': None, 'UpperNumberOfSeizures': None, 'TimePeriod': None, 'NumberOfTimePeriods': None, 'MonthDate': None, 'TimeSince_or_TimeOfEvent': 'Since', 'NumberOfSeizures': '0', 'PointInTime': 'LastClinic', 'FrequencyChange': None, 'YearDate': None, 'LowerNumberOfTimePeriods': None, 'UpperNumberOfTimePeriods': None, 'AgeUnit':








































































































